# Note, this *only* works for observing things in the future. Dealing with the pointing corrections in the analysis is not supported here.

# Set your observing times here

In [1]:
from datetime import datetime
tstart = '2017-06-18T00:00:00'
tend = '2017-06-20T00:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')                

# Step 1, get the nominal RA/Dec Position for Jupiter at the start

In [2]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../data')

ts = load.timescale()
planets = load('jup310.bsp')

astro_time = Time(tstart)
t = ts.from_astropy(astro_time)
jupiter, earth = planets['jupiter'], planets['earth']
astrometric = earth.at(t).observe(jupiter)
ra, dec, distance = astrometric.radec()

print(ra.to(u.deg), dec.to(u.deg))

print(tstart.timetuple().tm_year, tstart.timetuple().tm_yday)
print(tend.timetuple().tm_year, tend.timetuple().tm_yday)


[#################################] 100% deltat.data
[#################################] 100% deltat.preds
[#################################] 100% Leap_Second.dat
[#################################] 100% jup310.bsp


192.56390324798966 deg -3.907415328295961 deg
2017 169
2017 171


# Step 2: Go run the code that figured out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in ./orbit_engine that use the version that Karl already compiled for the nuops users.

Do:

`./get_latest_TLE.sh`

adjust the

`run_jupiter_test.sh`

script appropriately, then

`./run_jupiter_test.sh`

Check in the resulting jupiter occultation file into the repo, push it here, and them start below.


# Step 3: Parse the resulting occultation file:

In [3]:
file = '../orbit_engine/jupiter_169_171.occ'
from jupiter_planning import *
import jupiter_planning
import importlib
importlib.reload(jupiter_planning)
orbits = jupiter_planning.parse_occ(file)
orbits.head()

,visible,occulted
0,2017-06-18 00:25:18,2017-06-18 01:21:08
1,2017-06-18 02:01:58,2017-06-18 02:57:47
2,2017-06-18 03:38:38,2017-06-18 04:34:27
3,2017-06-18 05:15:17,2017-06-18 06:11:07
4,2017-06-18 06:51:57,2017-06-18 07:47:46


# Use SkyField to get the location of Jupiter for each orbit:

In [6]:
jupiter_planning.get_jupiter_radec(orbits, load_path = '../data', outfile='jupiter_169_171_pointing.txt', parallax_correction=True)

Total accumualted time 97139.0
